In [143]:
import numpy as np
# import cupy as cp
from matplotlib import pyplot as plt


use_gpu = False  # Cambiar a False si no tienes GPU o CuPy instalado
# np} = cp if use_gpu else np

# Consideraciones inciales.

### Funcion para construir la grilla inicial

In [144]:
# Función que construye la grilla inicial

def create_grid(d, grid_size, m, pattern='random'):
    """
    Crea una grilla n-dimensional
    
    Parámetros:
    - d: número de dimensiones
    - grid_size: tamaño de cada dimensión (int o lista de int)
    - m: valor máximo de estado (estados van de 0 a m + 1)
    - pattern: tipo de patrón inicial ('random', 'central', etc.)
    """
    # Convertir grid_size a una tupla si es un solo número
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
    
    # Crear la grilla según el patrón solicitado
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return np.random.randint(0, m+1, grid_size)
    
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
    
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = np.indices(grid_size)
        sum_indices = np.sum(indices, axis=0)
        return (sum_indices % 2) * m
    
    else:
        # Patrón predeterminado: valores 0
        return np.zeros(grid_size, dtype=int)
    


### Valores inciales

In [145]:
d = 3  # 3 dimensiones de la grilla
grid_size = 2  # Tamaño de los arreglos
m = 3  # Estados posibles por casilla
r = 1 # Radio de vecindad
grid = create_grid(d, grid_size, m, pattern='random') # Grilla inicial

t = 10 # Tiempo


In [146]:
grid

array([[[1, 1],
        [0, 0]],

       [[3, 0],
        [2, 2]]], dtype=int32)

In [147]:
def evolucionar_grilla(grilla, m, r):
    # Expandir la grilla
    grilla_expandida = np.pad(grilla, r, mode='wrap')
    
    # Crear kernel usando broadcasting
    kernel_shape = (2 * r + 1,) * grilla.ndim
    kernel = np.ones(kernel_shape, dtype=grilla.dtype)
    
    # Crear índices para centro sin bucles
    indices = np.indices(kernel_shape)
    centro = np.all(indices == r, axis=0)
    kernel[centro] = 0
    
    # Convolución para suma de vecinos
    from scipy.ndimage import convolve
    suma_vecinos = convolve(grilla_expandida, kernel, mode='constant', cval=0)
    
    # Recortar la parte válida usando indexación directa
    indices_slice = np.s_[r:-r, r:-r]  # Para 2D
    suma_vecinos_valida = suma_vecinos[indices_slice]
    
    # Aplicar reglas con broadcasting
    SM = m * np.sum(kernel)
    intervalos = np.linspace(0, SM, 4)
    
    nueva_grilla = np.copy(grilla)
    
    # Aplicar las reglas una por una en lugar de usar np.select
    mascara1 = (suma_vecinos_valida >= intervalos[0]) & (suma_vecinos_valida < intervalos[1])
    mascara2 = (suma_vecinos_valida >= intervalos[1]) & (suma_vecinos_valida < intervalos[2])
    mascara3 = suma_vecinos_valida >= intervalos[2]
    
    nueva_grilla[mascara1] -= 1
    nueva_grilla[mascara2] += 1
    nueva_grilla[mascara3] -= 1
    
    return np.clip(nueva_grilla, 0, m)

In [148]:
grilla_evolucionada = evolucionar_grilla(grid, m, r)

print(f"grilla original: \n", grid)

print(f"grilla final: \n", grilla_evolucionada)
# Visualizar la grilla antes y después
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(grid, cmap='viridis', interpolation='nearest')
axes[0].set_title("Grilla Inicial")
axes[1].imshow(grilla_evolucionada, cmap='viridis', interpolation='nearest')
axes[1].set_title("Grilla Evolucionada")

# plt.show()

IndexError: boolean index did not match indexed array along axis 2; size of axis is 2 but size of corresponding boolean axis is 4

### Funciones principales

### Pasos a seguir

1. for exterior -> paso del tiempo, por cada tick hace algo
    
    ¿Que hace? 

    a. Saca las otras celdas que pertenezcan a la vecindad

    b. Suma todos los valores 

    c. Se verifica con el intervalo si se + - o mantiene el valor de la celda
